In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from algorithm.kgvec2go_vectors_simple import run_method, load_gensim_model
from excludes.basic_excludes import (node_excl_yago_func, node_excl_extra,
                                     node_excl_owlthing_func, node_excl_wiki_func)
from loaders.loaders import *
from evaluation.split_dataset import split_on_seed_dataset
import json
import pickle
import os
from tqdm import tqdm


exclude_paths = [
    ["", "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "http://www.w3.org/2004/02/skos/core#Concept"]
]
relation_excludes = ["http://xmlns.com/foaf/0.1/primaryTopic",
                     "http://dbpedia.org/property/isCitedBy",
                     "http://dbpedia.org/ontology/wikiPageWikiLink",
                     "http://dbpedia.org/ontology/wikiPageWikiLinkText",
                     "http://dbpedia.org/property/wikiPageUsesTemplate",
                     'http://dbpedia.org/ontology/wikiPageRedirects',
                     'http://dbpedia.org/ontology/wikiPageOutDegree',
                     "http://dbpedia.org/ontology/abstract",
                     "http://www.w3.org/2000/01/rdf-schema#comment", ]

res_excludes_inside_point_relation = ["http://purl.org/linguistics/gold/hypernym",
                                      "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"]
res_excludes_inside_point_node = []
node_excludes_funcs = [node_excl_wiki_func, node_excl_owlthing_func, node_excl_yago_func, node_excl_extra]

SEED = 44
SAMPLE_SIZE = 4
verbose = 1

#Dataset load
#lc_quad_train = load_lc_quad_train()

lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)
splitted_lcquad = split_on_seed_dataset(lc_quad_train, SAMPLE_SIZE, SEED)

#splitted_lcquad = load_on_path_pickle("/home/kardosp/continuethelist/outputs/graphwalk_v2_output_sample4.pickle")

cwd = "/home/kardosp/continuethelist/dbpedia_vectors/kgvec2go"
word2vec_path = os.path.join(cwd, 'model.kv')
wv_model = load_gensim_model(word2vec_path)
missing_counter = 0

In [ ]:
# lcquad_output = run_on_dataset(splitted_lcquad, exclude_paths, relation_excludes, node_excludes_funcs)
import numpy as np
from scipy.spatial.distance import cdist

for i, record in tqdm(list(enumerate(splitted_lcquad[:1]))):
    print(i)
    if "output" in record:
        continue
    vectors = dict()
    missing = 0
    for item in record["seed"]:
        try:
            vectors[item] = list(wv_model[item])
        except Exception as e:
            missing += 1
    center_vector = np.mean(list(vectors.values()), axis=0)




    #point_dist = cdist([center_vector], list(vectors.values()), metric="euclidean")
    #threshold = np.max(point_dist)
    #eps = 1e-5
    #all_euclidean_dist = cdist([center_vector], wv_model.vectors, metric="euclidean")
    #closer_elements = list()
    #for index in all_euclidean_dist.argsort()[0]:
    #    if all_euclidean_dist[0][index] > threshold+eps:
    #        break
    #    closer_elements.append((index, all_euclidean_dist[0][index]))


    prediction = [wv_model.index_to_key[item[0]] for item in closer_elements]
    result = prediction
    record["output"] = result
    missing_counter += missing
    # TODO - evaluate

    with open("/home/kardosp/continuethelist/outputs/kgvec2go_v1_output_sample4.pickle", "wb") as f:
        pickle.dump(splitted_lcquad, f)



# TESTING small

In [4]:
from algorithm.kgvec2go_vectors_simple import ellipse_rectangle_dist, get_radius
import numpy as np
from scipy.spatial.distance import cdist

In [5]:
record = splitted_lcquad[1]
vectors = dict()
missing = 0
for item in record["seed"]:
    try:
        vectors[item] = list(wv_model[item])
    except Exception as e:
        missing += 1
center_vector = np.mean(list(vectors.values()), axis=0)

radius = get_radius(center_vector, list(vectors.values()))
all_l1ellipse_dist = cdist([center_vector], wv_model.vectors, metric=lambda x, y: ellipse_rectangle_dist(x, radius, y))

#prediction = [wv_model.index_to_key[item[0]] for item in closer_elements]
#result = prediction
#record["output"] = result
#missing_counter += missing

In [6]:
radius

array([0.4202354 , 0.3813632 , 0.97426176, 0.43779105, 0.5417757 ,
       1.0694426 , 0.30470932, 0.6588133 , 0.73345053, 1.6430582 ,
       0.95992863, 1.0176697 , 1.019561  , 0.84902245, 0.4403092 ,
       1.0942907 , 0.49112612, 1.0314772 , 0.15219314, 0.16184996,
       0.14828266, 0.32938522, 0.5592884 , 0.9363576 , 0.2766841 ,
       0.11928315, 0.37348533, 0.7520633 , 0.36028004, 0.7661077 ,
       0.21373959, 0.4735586 , 0.4585205 , 0.33553088, 0.54396194,
       0.42406794, 0.62913555, 0.3926924 , 0.69289625, 0.37293473,
       0.8079519 , 0.6863488 , 0.20818959, 0.5004904 , 0.5614491 ,
       0.57291067, 0.9827553 , 0.6031111 , 0.26760057, 0.7260299 ,
       0.43058836, 0.30454913, 0.150023  , 0.37361434, 0.5660397 ,
       0.97249043, 0.597338  , 0.9799927 , 0.29632777, 0.3780715 ,
       0.11129463, 0.5933902 , 0.3385514 , 0.61557776, 1.0071933 ,
       0.29305875, 0.66120726, 0.31413305, 0.3652076 , 0.53897774,
       0.689991  , 0.22227061, 0.640576  , 0.5317434 , 0.63811

In [7]:
all_l1ellipse_dist

array([[213.08602905, 570.47650146, 262.80407715, ...,   1.14424789,
          1.23790288,   1.17283082]])

In [8]:
sorted_distances = all_l1ellipse_dist.argsort()

In [9]:
sorted_distances.shape

(1, 5818450)

In [10]:
for item in sorted_distances[0][:10]:
    print(all_l1ellipse_dist[0][item])

0.8980002403259277
0.9169031381607056
0.9207807183265686
0.92329341173172
0.9239602088928223
0.9281923770904541
0.9295969605445862
0.9342576265335083
0.9343723654747009
0.9346258044242859


# Ellipse solution

In [13]:
from copy import copy

In [17]:
record = splitted_lcquad[1]
vectors = dict()
missing = 0
for item in record["seed"]:
    try:
        vectors[item] = list(wv_model[item])
    except Exception as e:
        missing += 1
center_vector = np.mean(list(vectors.values()), axis=0)

wv_vectors = copy(wv_model.vectors)
radius = get_radius(center_vector, list(vectors.values()))
#all_l1ellipse_dist = cdist([center_vector], wv_model.vectors, metric=lambda x, y: ellipse_rectangle_dist(x, radius, y))

#prediction = [wv_model.index_to_key[item[0]] for item in closer_elements]
#result = prediction
#record["output"] = result
#missing_counter += missing

In [15]:
type(wv_vectors)

numpy.memmap

In [24]:
wv_vectors_sample = wv_vectors[:10, :5]
wv_vectors_sample.shape

(10, 5)

In [25]:
wv_vectors_sample

array([[ 0.23574328,  0.30461386, -0.6003029 , -0.60192037, -0.22962554],
       [ 0.22316432,  0.16041347, -0.3710722 , -0.4242497 , -0.26547986],
       [ 0.12492717,  0.30506092, -0.7972921 , -0.50669885, -0.2456873 ],
       [ 0.35329735, -0.13531597, -0.09536346, -0.5517705 , -0.5212886 ],
       [ 0.2871616 , -0.08491012, -0.23584919, -0.40193594, -0.57611215],
       [ 0.45754758,  0.35635844, -0.60824203, -0.96065724, -0.30236956],
       [ 0.3462565 , -0.07093415, -0.5527556 , -0.6642382 , -0.14996691],
       [ 0.21384846, -0.02370554,  0.44629025, -0.64765763, -0.9027733 ],
       [ 0.43125504, -0.15474823, -0.04339918, -0.7359958 , -0.7884353 ],
       [ 0.09070162,  0.16193217, -0.59742916, -0.6666378 , -0.25581402]],
      dtype=float32)

In [26]:
wv_vectors_sample / radius[:5]

array([[ 0.5609791 ,  0.79875   , -0.6161618 , -1.3749033 , -0.42383876],
       [ 0.531046  ,  0.42063174, -0.38087526, -0.96906894, -0.49001804],
       [ 0.29727903,  0.7999223 , -0.81835514, -1.1573988 , -0.45348528],
       [ 0.84071296, -0.35482177, -0.09788279, -1.2603513 , -0.9621852 ],
       [ 0.68333507, -0.22264896, -0.2420799 , -0.91809994, -1.0633776 ],
       [ 1.0887887 ,  0.93443316, -0.6243107 , -2.194328  , -0.5581084 ],
       [ 0.8239584 , -0.18600154, -0.5673584 , -1.5172495 , -0.27680627],
       [ 0.50887775, -0.06216   ,  0.45808044, -1.4793762 , -1.6663231 ],
       [ 1.0262226 , -0.4057765 , -0.04454571, -1.6811578 , -1.4552798 ],
       [ 0.21583526,  0.42461404, -0.61321217, -1.5227305 , -0.47217697]],
      dtype=float32)